<center>
<img src="https://www.iybssd2022.org/wp-content/uploads/ASAQ.jpg" width="150"/> 
</center>

        
<center>
<h1><font color= "blue" size="+2">ASAQ Python Data Analysis Courses</font></h1>
</center>

---

<center><h1><font color="blue" size="+2">Manipulating Cape Town Data</font></h1></center>

## <font color="red">Objectives</font>

We want to:

- Read a file containing measurements around the Cape Town area and do manipualtion and viualization

## <font color="red">Required modules/packages</font>

- __Matplotlib__: for basic plots.
- __Pandas__: Manipulation and exploratory data analysis of tabular data.
- __Shapely__: For manipulation and analysis of planar geometric objects
- __GeosPandas__: Combines the capabilities of Pandas and Shapely for geospatial operations
- __Plotly__: Perform interactive visualization.

In [ ]:
try:
    import google.colab
    print("Running in Google Colab")
except:
    print("Not running in Google Colab")
else:
    print("Installing modules in Google Colab")
    !sudo apt-get update && apt-get install -y libspatialindex-dev
    !pip install rtree
    !pip install geopandas
    !pip install mapclassify
    !pip install plotly

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from pathlib import Path

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.ticker as mticker
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable

In [ ]:
import pandas as pd

In [ ]:
import geopandas as gpd

In [ ]:
from shapely import geometry as shpgeom
from shapely import wkt as shpwkt

In [ ]:
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
pio.templates.default = "plotly_white"

__Read the file with Cape Town data__

In [ ]:
file_name = "ap_cape_town.xlsx"
data_url = f"https://github.com/JulesKouatchou/asaq_py/raw/main/sample_data/{file_name}"
#data_url = "/".join(["../sample_data", file_name])

In [ ]:
df = pd.read_excel(data_url, 
                   sheet_name="Feuil1",
                  parse_dates={'t': [0]}
                  )
df

In [ ]:
df.info()

__Find unique values in each column__

In [ ]:
for name in df.columns:
    print(f"{name}: {df[name].unique()}")

__The following columns ahve one value only and can be removed:__
- `t`
- `City`
- `Source Name`
- `Country Label`

In [ ]:
df.drop(columns=['t', 'City', 'Source Name', 'Country Label'], inplace=True)
df

__Convert the `Last Updated` column into a datetime object__

In [ ]:
df['t'] = pd.to_datetime(df['Last Updated'])

__Remove the `Last Updated` column__

In [ ]:
df.drop(columns=['Last Updated'], inplace=True)
df

__Make the time as the index of the DataFrame__

In [ ]:
df.set_index('t', inplace=True)

__Split the `Coordinates` column into two columns of `latitude` and `longitude`__

In [ ]:
def spilt_column(row):
    coord = row['Coordinates'].strip().split(',')
    return [float(coord[0]), float(coord[1])]

In [ ]:
df[['latitude', 'longitude']] = df.apply(spilt_column, axis=1, result_type='expand')
df

__Remove the `Coordinates` column__

In [ ]:
df.drop(columns=['Coordinates'], inplace=True)
df

__Basic plot of the locations__

In [ ]:
ax = df.plot.scatter(x="longitude", y="latitude", color="red")
# Add text annotations to each point
for i, row in df.iterrows():
    ax.annotate(row['Location'], (row['longitude'], row['latitude']), fontsize=6, 
                textcoords="offset points", xytext=(10,-5), ha='center')

In [ ]:
fig = px.scatter(df, x="longitude", y="latitude", text="Location")
fig.update_traces(textposition='middle right')
#fig.update_layout(uniformtext_minsize=4)
fig.show()

__Zip the `latitude` and `longitude` together to create a new column named `Coordinates`__

In [ ]:
df["Coordinates"] = list(zip(df.longitude, df.latitude))
df

__Convert the `Coordinates` column into a Shapely Point object__

In [ ]:
df["Coordinates"] = df["Coordinates"].apply(shpgeom.Point)
df

__Create a GeoPandas GeoDataFrame__

In [ ]:
gdf = gpd.GeoDataFrame(df, geometry="Coordinates")
gdf.head()

__Plot the locations__

In [ ]:
fig, axes = plt.subplots(figsize=(18, 6))

gdf.plot(ax=axes, color='red', markersize=3);

__Obtain the map of the Cape Town area__

- [South Africa, Cape Town (Metropolitan Council, 1998)](https://geodata.libraries.mit.edu/record/gismit:SF_WC_F8METROCOUNCIL_1998)
- Geospatial coordinates
   - Bounding Box: BBOX (22.735743, 30.197877, -29.963087, -34.213643)
   - Geometry: BBOX (22.735743, 30.197877, -29.963087, -34.213643)

In [ ]:
ct_url = "https://cdn.libraries.mit.edu/geo/public/SF_WC_F8METROCOUNCIL_1998.zip"

In [ ]:
ct_zip_fname = Path(ct_url).name
ct_zip_dir = Path(ct_url).stem
ct_zip_dir

In [ ]:
import urllib.request
import zipfile

In [ ]:
urllib.request.urlretrieve(ct_url, ct_zip_fname)

In [ ]:
with zipfile.ZipFile(ct_zip_fname, "r") as zip_ref:
    zip_ref.extractall("./")

__Read and plot the Cape Town shapefile__

In [ ]:
ct_gdf = gpd.read_file(ct_zip_dir)
ct_gdf

In [ ]:
type(ct_gdf)

In [ ]:
ct_gdf.plot(figsize=(9,12));

__Overlay the data over the Cape Town map__

In [ ]:
base = ct_gdf.boundary.plot(figsize=(5,28));
gdf.plot(ax=base, marker='o', color='red', markersize=5)

In [ ]:
m = ct_gdf.explore()

gdf.explore(
    m=m,
    column="Pollutant",
    tiles="CartoDB positron",
    cmap="viridis_r",
    style_kwds={"style_function":lambda x: {"radius":x["properties"]["Value"]}}
)
